## MAli v1.31 Alignment + Scoring

#### Imports

In [1]:
import os
import shutil
import subprocess
import time
import pandas as pd

In [2]:
from wrapped_scorer import WrappedScorer

#### MAli v1.3

In [3]:
SEED = 9032025
ITERATIONS = 200

In [4]:
ALIGNER_NAME = "MAli-v1.31"
ALIGNER_PATH = "aligners/MAli-v1.31/MAli.exe"
OUTPUT_FOLDER = "aligners/MAli-v1.31/output"

In [5]:
# creating empty output folder
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)
os.makedirs(OUTPUT_FOLDER)

#### BALIS-1

In [6]:
DATASET_NAME = "BALIS-1"
DATASET_FOLDER = "datasets/BALIS-1"
INPUT_FOLDER = f"{DATASET_FOLDER}/in"
REFERENCES_FOLDER = f"{DATASET_FOLDER}/ref"

In [7]:
# checking that testcases have been found
testcases = os.listdir(INPUT_FOLDER)
print(testcases[:10])

['BB11004', 'BB11005', 'BB11008', 'BB11009', 'BB11011', 'BB11014', 'BB11015', 'BB11016', 'BB11018', 'BB11019']


#### QScore

In [8]:
SCORER_PATH = "scorers/QScore/qscore.exe"

In [9]:
scorer = WrappedScorer(SCORER_PATH)

#### Performing Alignment

In [10]:
def perform_alignment_and_record_time(filename):

    input_path = f"{INPUT_FOLDER}/{filename}"
    output_path = f"{OUTPUT_FOLDER}/{filename}"
    command = f"{ALIGNER_PATH} -input {input_path} -output {output_path} -seed {SEED} -iterations {ITERATIONS}"

    start_time = time.perf_counter()
    
    subprocess.run(command)

    end_time = time.perf_counter()
    
    time_in_milliseconds = (end_time - start_time) * 1000
    time_in_milliseconds = round(time_in_milliseconds, 0)

    return int(time_in_milliseconds)

In [11]:
def score_quality_of_produced_alignment(filename):

    test_path = f"{OUTPUT_FOLDER}/{filename}.faa"
    reference_path = f"{REFERENCES_FOLDER}/{filename}"
    score = scorer.score_testcase(test_path, reference_path)

    return score

In [12]:
def record_performance_on_testcase(filename):

    time_taken = perform_alignment_and_record_time(testcase)
    score = score_quality_of_produced_alignment(testcase)


    return f"{ALIGNER_NAME},{DATASET_NAME},{filename},{score},{time_taken}"
    

In [13]:
HEADER = "aligner,dataset,testcase,Q_score,time_elapsed_ms"
RECORDS = []
RECORDS.append(HEADER)

In [14]:
for testcase in testcases:
    record = record_performance_on_testcase(testcase)
    RECORDS.append(record)
    print(record)

MAli-v1.31,BALIS-1,BB11004,0.0851,10241
MAli-v1.31,BALIS-1,BB11005,0.0512,14767
MAli-v1.31,BALIS-1,BB11008,0.245,13136
MAli-v1.31,BALIS-1,BB11009,0.0917,8319
MAli-v1.31,BALIS-1,BB11011,0.115,6031
MAli-v1.31,BALIS-1,BB11014,0.614,14710
MAli-v1.31,BALIS-1,BB11015,0.437,7516
MAli-v1.31,BALIS-1,BB11016,0.00779,16754
MAli-v1.31,BALIS-1,BB11018,0.0718,23650
MAli-v1.31,BALIS-1,BB11019,0.278,9949
MAli-v1.31,BALIS-1,BB11021,0.231,3551
MAli-v1.31,BALIS-1,BB11022,0.0505,5957
MAli-v1.31,BALIS-1,BB11025,0.0526,2545
MAli-v1.31,BALIS-1,BB11027,0.0517,9196
MAli-v1.31,BALIS-1,BB11029,0.176,3837
MAli-v1.31,BALIS-1,BB11031,0.0691,17059
MAli-v1.31,BALIS-1,BB11033,0.0566,6421
MAli-v1.31,BALIS-1,BB11038,0.199,18709
MAli-v1.31,BALIS-1,BB12005,0.837,5030
MAli-v1.31,BALIS-1,BB12013,0.93,16053
MAli-v1.31,BALIS-1,BB12014,0.84,2194
MAli-v1.31,BALIS-1,BB12016,0.658,2794
MAli-v1.31,BALIS-1,BB12021,0.91,2185
MAli-v1.31,BALIS-1,BB12024,0.875,5588
MAli-v1.31,BALIS-1,BB12026,0.837,5879
MAli-v1.31,BALIS-1,BB12028,0.523,

In [15]:
print(RECORDS[0])
print(RECORDS[-1])

aligner,dataset,testcase,Q_score,time_elapsed_ms
MAli-v1.31,BALIS-1,BB50014,0.737,14362


#### Writing Records to CSV

In [16]:
RESULTS_FILENAME = f"sbench_{ALIGNER_NAME}_on_{DATASET_NAME}.csv"
RESULTS_FILEPATH = f"results/{RESULTS_FILENAME}"

In [17]:
with open(RESULTS_FILEPATH, "w") as file:
    for record in RECORDS:
        file.write(record)
        file.write("\n")

In [18]:
print(f"Results written to: {RESULTS_FILEPATH}")

Results written to: results/sbench_MAli-v1.31_on_BALIS-1.csv


#### Previewing Results

In [19]:
df = pd.read_csv(RESULTS_FILEPATH)
df.head()

,aligner,dataset,testcase,Q_score,time_elapsed_ms
0,MAli-v1.31,BALIS-1,BB11004,0.0851,10241
1,MAli-v1.31,BALIS-1,BB11005,0.0512,14767
2,MAli-v1.31,BALIS-1,BB11008,0.2450,13136
3,MAli-v1.31,BALIS-1,BB11009,0.0917,8319
4,MAli-v1.31,BALIS-1,BB11011,0.1150,6031
